In [3]:
import warnings
from collections import defaultdict

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

DIR_PATH = "./Datasets"
warnings.filterwarnings("ignore")
plt.rcParams["figure.figsize"] = (28, 10)

### Data Exploration & Feature Engineering

  - https://dcc.icgc.org/releases/PCAWG/evolution_and_heterogeneity
  - https://www.science.org/doi/full/10.1126/science.abl9283 (table S6)
  - https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6872491/ (table S2)
  - https://www.nature.com/articles/s43018-021-00200-0 (table S9)

In [4]:
df = pd.read_csv(f"{DIR_PATH}/phg_clincal_activities_and_signatures.csv").set_index("Unnamed: 0")
df.head()

,pcawg_class,WGD,num_subclones,tumour_type,reported_sex,donor_survival_time,donor_age_at_diagnosis,tumour_stage,tumour_grade,first_therapy_type,...,SBS85,SBS86,SBS87,SBS88,SBS89,SBS90,SBS91,SBS92,SBS93,SBS94
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
0009b464-b376-4fbc-8a56-da538269a02f,Ovary-AdenoCA,True,1.0,Recurrent,Female,1972.0,54.0,NaN,NaN,NaN,...,0.004170,0.032632,0.009180,0.004538,0.015247,0.023516,0.001096,0.001524,0.041314,0.074747
003819bc-c415-4e76-887c-931d60ed39e7,CNS-PiloAstro,False,1.0,Primary,Female,244.0,4.0,NaN,1,NaN,...,0.011223,0.000018,0.043297,0.002587,0.000110,0.001679,0.000330,0.008414,0.012260,0.010864
0040b1b6-b07a-4b6e-90ef-133523eaf412,Liver-HCC,False,1.0,Primary,Male,1905.0,73.0,NaN,G2,no treatment,...,0.010256,0.001064,0.000472,0.006128,0.014017,0.003671,0.002160,0.015290,0.010472,0.012708
00493087-9d9d-40ca-86d5-936f1b951c93,CNS-Oligo,False,1.0,Primary,Female,NaN,47.0,NaN,NaN,NaN,...,0.034194,0.007644,0.066593,0.003875,0.008409,0.001092,0.004163,0.013443,0.012461,0.046001
00508f2b-36bf-44fc-b66b-97e1f3e40bfa,Panc-Endocrine,False,2.0,Primary,Female,7.0,59.0,T2N1MX,2 - Moderately differentiated,NaN,...,0.002131,0.006881,0.014572,0.005315,0.034945,0.000375,0.024285,0.001300,0.002455,0.023321


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18947 entries, 0009b464-b376-4fbc-8a56-da538269a02f to CPCT02220071T
Columns: 119 entries, pcawg_class to SBS94
dtypes: float64(85), int64(1), object(33)
memory usage: 17.3+ MB


In [5]:
SBS_COLS = [col for col in df.columns if "SBS" in col]
DATA_COLS = [col for col in df.columns if "SBS" not in col]
OBJECT_COLS = df.select_dtypes(include=['object']).columns
TARGET = "tumour_type"
DATA_COLS

['pcawg_class',
 'WGD',
 'num_subclones',
 'tumour_type',
 'reported_sex',
 'donor_survival_time',
 'donor_age_at_diagnosis',
 'tumour_stage',
 'tumour_grade',
 'first_therapy_type',
 'first_therapy_response',
 'specimen_donor_treatment_type',
 'smoking_history',
 'tobacco_smoking_intensity',
 'alcohol_history',
 'alcohol_history_intensity',
 'ancestry_primary',
 'ancestry_primary_contribution',
 'cohort',
 'organ',
 'is_pretreated',
 'had_radiotherapy',
 'is_hypermutated',
 'HRD',
 'biopsy_site',
 'had_other_treatment',
 'had_chemotherapy',
 'had_hormone_therapy',
 'had_targeted_therapy',
 'had_immunotherapy',
 'treatment_platinum',
 'treatment_5FU',
 'genetic_immune_escape',
 'primary_tumour_location',
 'cancer_subtype',
 'age_at_biopsy',
 'MSIseq',
 'MMRDetect',
 'qc_pass',
 'stage_coarse',
 'TMB']

In [6]:
df[DATA_COLS].describe().T

,count,mean,std,min,25%,50%,75%,max
num_subclones,2778.0,1.030238,0.730887,0.000000,1.000000,1.000000,1.0000,4.000000e+00
donor_survival_time,1754.0,1223.050741,1333.317455,0.000000,413.000000,810.000000,1460.7500,1.095800e+04
donor_age_at_diagnosis,2650.0,56.165660,19.249394,1.000000,48.000000,60.000000,70.0000,9.000000e+01
tobacco_smoking_intensity,150.0,27.566667,26.616949,0.000000,0.000000,21.000000,50.0000,8.600000e+01
ancestry_primary_contribution,2569.0,0.958425,0.086477,0.340681,0.961663,0.994459,0.9996,9.996000e-01
age_at_biopsy,2004.0,61.261477,11.329076,19.000000,54.000000,63.000000,69.0000,8.900000e+01
stage_coarse,530.0,2.509434,0.970698,1.000000,2.000000,3.000000,3.0000,4.000000e+00
TMB,18947.0,23365.237663,93283.930542,21.000000,3460.000000,6878.000000,15297.0000,2.745037e+06


In [7]:
df[DATA_COLS].isna().sum(axis = 0)

pcawg_class                      16169
WGD                              14149
num_subclones                    16169
tumour_type                      12752
reported_sex                     14149
donor_survival_time              17193
donor_age_at_diagnosis           16297
tumour_stage                     17456
tumour_grade                     17475
first_therapy_type               17962
first_therapy_response           18393
specimen_donor_treatment_type    17179
smoking_history                  17640
tobacco_smoking_intensity        18797
alcohol_history                  17730
alcohol_history_intensity        17730
ancestry_primary                 16378
ancestry_primary_contribution    16378
cohort                               0
organ                                0
is_pretreated                    16927
had_radiotherapy                 17038
is_hypermutated                  16934
HRD                              16927
biopsy_site                      16935
had_other_treatment      

Quite large numbers of NaN's. Discarding those is a bad decision, a lot of data will be lost

In [9]:
for col in DATA_COLS:
    print(f"Column - {col}")
    print(f"Unique values - {df[col].unique()}")
    print(f"NaN percentage - {df[col].isna().sum() / df[col].shape[0]}")
    print()

Column - pcawg_class
Unique values - ['Ovary-AdenoCA' 'CNS-PiloAstro' 'Liver-HCC' 'CNS-Oligo' 'Panc-Endocrine'
 'Kidney-RCC' 'Prost-AdenoCA' 'Thy-AdenoCA' 'ColoRect-AdenoCA'
 'Lymph-BNHL' 'Uterus-AdenoCA' 'Breast-AdenoCA' 'Lung-AdenoCA'
 'Panc-AdenoCA' 'Eso-AdenoCA' 'Head-SCC' 'CNS-Medullo' 'CNS-GBM'
 'Bone-Leiomyo' 'Cervix-SCC' 'Skin-Melanoma' 'Lymph-CLL' 'Kidney-ChRCC'
 'Stomach-AdenoCA' 'Lung-SCC' 'Bladder-TCC' 'Lymph-NOS' 'Myeloid-AML'
 'Biliary-AdenoCA' 'Breast-LobularCA' 'Cervix-AdenoCA' 'Bone-Osteosarc'
 'Breast-DCIS' 'Myeloid-MPN' 'Myeloid-MDS' 'Bone-Cart' 'Bone-Epith' nan]
NaN percentage - 0.8533804823982688

Column - WGD
Unique values - [True False nan]
NaN percentage - 0.7467672982530216

Column - num_subclones
Unique values - [ 1.  2.  0.  3.  4. nan]
NaN percentage - 0.8533804823982688

Column - tumour_type
Unique values - ['Recurrent' 'Primary' 'Metastatic' 'Cell' nan]
NaN percentage - 0.6730353090198976

Column - reported_sex
Unique values - ['Female' 'Male' nan]
NaN per

Create column which indicates number of NaN's

In [10]:
df["Number of NaN's"] = df.isna().sum(axis = 1)
df.head()

,pcawg_class,WGD,num_subclones,tumour_type,reported_sex,donor_survival_time,donor_age_at_diagnosis,tumour_stage,tumour_grade,first_therapy_type,...,SBS86,SBS87,SBS88,SBS89,SBS90,SBS91,SBS92,SBS93,SBS94,Number of NaN's
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
0009b464-b376-4fbc-8a56-da538269a02f,Ovary-AdenoCA,True,1.0,Recurrent,Female,1972.0,54.0,NaN,NaN,NaN,...,0.032632,0.009180,0.004538,0.015247,0.023516,0.001096,0.001524,0.041314,0.074747,27
003819bc-c415-4e76-887c-931d60ed39e7,CNS-PiloAstro,False,1.0,Primary,Female,244.0,4.0,NaN,1,NaN,...,0.000018,0.043297,0.002587,0.000110,0.001679,0.000330,0.008414,0.012260,0.010864,23
0040b1b6-b07a-4b6e-90ef-133523eaf412,Liver-HCC,False,1.0,Primary,Male,1905.0,73.0,NaN,G2,no treatment,...,0.001064,0.000472,0.006128,0.014017,0.003671,0.002160,0.015290,0.010472,0.012708,23
00493087-9d9d-40ca-86d5-936f1b951c93,CNS-Oligo,False,1.0,Primary,Female,NaN,47.0,NaN,NaN,NaN,...,0.007644,0.066593,0.003875,0.008409,0.001092,0.004163,0.013443,0.012461,0.046001,29
00508f2b-36bf-44fc-b66b-97e1f3e40bfa,Panc-Endocrine,False,2.0,Primary,Female,7.0,59.0,T2N1MX,2 - Moderately differentiated,NaN,...,0.006881,0.014572,0.005315,0.034945,0.000375,0.024285,0.001300,0.002455,0.023321,25


Fill NaN's

For objects - set to `"No value"`, for numeric - `.median()`

In [11]:
for col in DATA_COLS:
    if col in OBJECT_COLS:
        df[col] = df[col].fillna("No value")
    else:
        df[col] = df[col].fillna(df[col].median())

df.isna().sum()

pcawg_class        0
WGD                0
num_subclones      0
tumour_type        0
reported_sex       0
                  ..
SBS91              0
SBS92              0
SBS93              0
SBS94              0
Number of NaN's    0
Length: 120, dtype: int64

Filter dataframe by target 

In [12]:
df = df[df[TARGET].isin(['Primary', 'Metastatic'])]
df.head()

,pcawg_class,WGD,num_subclones,tumour_type,reported_sex,donor_survival_time,donor_age_at_diagnosis,tumour_stage,tumour_grade,first_therapy_type,...,SBS86,SBS87,SBS88,SBS89,SBS90,SBS91,SBS92,SBS93,SBS94,Number of NaN's
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
003819bc-c415-4e76-887c-931d60ed39e7,CNS-PiloAstro,False,1.0,Primary,Female,244.0,4.0,No value,1,No value,...,0.000018,0.043297,0.002587,0.000110,0.001679,0.000330,0.008414,0.012260,0.010864,23
0040b1b6-b07a-4b6e-90ef-133523eaf412,Liver-HCC,False,1.0,Primary,Male,1905.0,73.0,No value,G2,no treatment,...,0.001064,0.000472,0.006128,0.014017,0.003671,0.002160,0.015290,0.010472,0.012708,23
00493087-9d9d-40ca-86d5-936f1b951c93,CNS-Oligo,False,1.0,Primary,Female,810.0,47.0,No value,No value,No value,...,0.007644,0.066593,0.003875,0.008409,0.001092,0.004163,0.013443,0.012461,0.046001,29
00508f2b-36bf-44fc-b66b-97e1f3e40bfa,Panc-Endocrine,False,2.0,Primary,Female,7.0,59.0,T2N1MX,2 - Moderately differentiated,No value,...,0.006881,0.014572,0.005315,0.034945,0.000375,0.024285,0.001300,0.002455,0.023321,25
005794f1-5a87-45b5-9811-83ddf6924568,Kidney-RCC,False,1.0,Primary,Female,2112.0,62.0,No value,1,No value,...,0.002754,0.006760,0.008834,0.040637,0.002801,0.003005,0.011455,0.008315,0.079724,26


Percentages for each tumour type

In [ ]:
df[TARGET].value_counts() / df.shape[0]

Metastatic    0.577479
Primary       0.422521
Name: tumour_type, dtype: float64

In [ ]:
plt.rcParams["figure.figsize"] = (30, 30)
sns.pairplot(data = df[DATA_COLS], hue=TARGET, markers=["o", "s"])

In [ ]:
plt.rcParams["figure.figsize"] = (15, 15)
for col in DATA_COLS:
    if col in OBJECT_COLS:
        sns.countplot(data=df, x=col, hue=TARGET)
    else:
        sns.histplot(data=df, x=col, hue=TARGET)
    plt.show()

After substituting NaN's, there are clear relationships found in the dataframe between target and other categorical variables.
Numerical columns also show significant difference between the classes

Now let's look at signatures columns:

In [ ]:
plt.rcParams["figure.figsize"] = (15, 15)
for col in SBS_COLS:
    sns.histplot(data=df, x=col, hue=TARGET)
    plt.show()

Hard to make a clear distinguish between two classes - `Primary` is more right skewed for some mutations and vice versa, but overall they are very similar

Columns that need to be encoded(dummies) and LabelEncoding

In [ ]:
dummies_to_encode = [col for col in (list(OBJECT_COLS) + ["num_subclones", "stage_coarse"]) if col != TARGET]
label_to_encode = list(OBJECT_COLS)
df_dummies = pd.get_dummies(df, columns = dummies_to_encode)
df_dummies.shape

(6163, 746)

In [ ]:
df_label = df.copy(deep = True)
encoders_dict = defaultdict(LabelEncoder)

for col in label_to_encode:
    df_label[col] = df[col].apply(str)
    df_label[col] = encoders_dict[col].fit_transform(df_label[col])

df_label.head()

,pcawg_class,WGD,num_subclones,tumour_type,reported_sex,donor_survival_time,donor_age_at_diagnosis,tumour_stage,tumour_grade,first_therapy_type,...,SBS86,SBS87,SBS88,SBS89,SBS90,SBS91,SBS92,SBS93,SBS94,Number of NaN's
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
003819bc-c415-4e76-887c-931d60ed39e7,12,0,1.0,1,0,244.0,4.0,29,0,0,...,0.000018,0.043297,0.002587,0.000110,0.001679,0.000330,0.008414,0.012260,0.010864,23
0040b1b6-b07a-4b6e-90ef-133523eaf412,20,0,1.0,1,1,1905.0,73.0,29,21,4,...,0.001064,0.000472,0.006128,0.014017,0.003671,0.002160,0.015290,0.010472,0.012708,23
00493087-9d9d-40ca-86d5-936f1b951c93,11,0,1.0,1,0,810.0,47.0,29,37,0,...,0.007644,0.066593,0.003875,0.008409,0.001092,0.004163,0.013443,0.012461,0.046001,29
00508f2b-36bf-44fc-b66b-97e1f3e40bfa,32,0,2.0,1,0,7.0,59.0,52,3,0,...,0.006881,0.014572,0.005315,0.034945,0.000375,0.024285,0.001300,0.002455,0.023321,25
005794f1-5a87-45b5-9811-83ddf6924568,19,0,1.0,1,0,2112.0,62.0,29,0,0,...,0.002754,0.006760,0.008834,0.040637,0.002801,0.003005,0.011455,0.008315,0.079724,26


In [ ]:
df_dummies[TARGET] = encoders_dict[TARGET].transform(df_dummies[TARGET])

### Modeling

In [ ]:
from sklearn.decomposition import PCA
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, train_test_split, cross_validate, GridSearchCV
from sklearn.metrics import f1_score, roc_auc_score, precision_score, recall_score, accuracy_score, make_scorer

RANDOM_STATE = 100
MODELS = [
    GradientBoostingClassifier, RandomForestClassifier, LogisticRegression,
    KNeighborsClassifier, SVC, LinearSVC, LGBMClassifier, XGBClassifier
]
cv = StratifiedKFold(n_splits = 5, shuffle = True, random_state = RANDOM_STATE)
scorers = {
  'roc_auc': make_scorer(roc_auc_score),
  'accuracy': make_scorer(accuracy_score),
  'f1': make_scorer(f1_score),
  'precision': make_scorer(precision_score),
  'recall': make_scorer(recall_score),
}

In [ ]:
X, y = df_dummies.drop(TARGET, axis=1), df_dummies[TARGET]
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle = True, stratify = y)
X_train.shape, X_test.shape

((4622, 745), (1541, 745))

In [ ]:
reducer = PCA(n_components = 1)
X_truncated = reducer.fit_transform(X_test)
X_truncated = X_truncated.reshape(1, -1)[0]
reducer.explained_variance_ratio_.sum()

0.9999425548902304

In [ ]:
def model_metrics(y_true, y_predicted):
    print(f"Accuracy : {accuracy_score(y_true, y_predicted)}")
    print(f"Precision : {precision_score(y_true, y_predicted)}")
    print(f"Recall : {recall_score(y_true, y_predicted)}")
    print(f"F1 Score : {f1_score(y_true, y_predicted)}")
    print(f"AUC Score : {roc_auc_score(y_true, y_predicted)}")

def scatter_predicted_vs_true(x, y_true, y_predicted):
    fig, ax = plt.subplots()
    ax.scatter(x, y_true, c='green')
    ax.scatter(x, y_predicted, c='red')
    plt.show()

def cv_model(model_type, model_name, X_train, y_train, **kwargs):
    model = model_type(**kwargs)
    cv_results = cross_validate(model, X_train, y_train, scoring = scorers, cv = cv)
    print(f"Model - {model_name}, Results - {cv_results['test_roc_auc']}, Mean - {cv_results['test_roc_auc'].mean()}")
    score = cv_results['test_roc_auc'].mean()
    if np.isnan(score):
        score = 0
    return score

In [ ]:
def fit_cv(X_train, y_train):
    scores = dict()
    model_names = [str(model_cls.__name__) for model_cls in MODELS]

    for model_cls in MODELS:
        model_name = str(model_cls.__name__)
        scores[model_name] = cv_model(model_cls, model_name, X_train, y_train)
    scores = dict(sorted(scores.items(), key=lambda item: item[1]))

    best_model_name = None
    for i, model_name in enumerate(scores):
        print(f"{i + 1}. {model_name} - {scores[model_name]}")
        best_model_name = model_name
    best_model_cls = MODELS[model_names.index(best_model_name)]
    
    return best_model_cls

best_model = fit_cv(X_train, y_train)
best_model

In [ ]:
grid = {
    'n_estimators': list(range(60, 221, 40)) + [500, 1000], 
    'learning_rate': [0.05, 0.1, 0.3], 
    'reg_alpha': [0, 0.5, 2],
    'reg_lambda': [0, 0.5, 2],
    'colsample_bytree': [0.4, 0.6, 0.8, 1], 
    'subsample': [0.6, 0.8, 1]
}
fit_kwargs = {"eval_metric" : "auc", "eval_set": [(X_train, y_train), (X_test, y_test)], "early_stopping_rounds": 20}
cv_estimator = GridSearchCV(best_model(), grid, cv = cv, scoring = scorers['roc_auc'])
cv_estimator.fit(X_train, y_train, **fit_kwargs)
cv_estimator.score(X_test, y_test)

In [ ]:
cv_estimator.best_params_

In [ ]:
y_pred = cv_estimator.predict(X_test)
model_metrics(y_test, y_pred)
scatter_predicted_vs_true(X_truncated, y_test, y_pred)

Final model 

In [ ]:
best_params = {
    'colsample_bytree': 0.4,
    'learning_rate': 0.05,
    'n_estimators': 60,
    'reg_alpha': 0,
    'reg_lambda': 0,
    'subsample': 0.6
}
fit_kwargs = {
    "eval_metric" : "auc", 
    "eval_set": [(X_train, y_train), (X_test, y_test)], 
    "early_stopping_rounds": 20
}
model = LGBMClassifier(**best_params)
model.fit(X_train, y_train, **fit_kwargs)
model.score(X_test, y_test)

Feature importances

In [ ]:
feature_imp = sorted([(imp, col) for imp, col in zip(model.feature_importances_, X.columns) if imp > 0])
feature_imp = pd.DataFrame(feature_imp, columns=['Value','Feature'])
plt.figure(figsize=(28, 30))
sns.barplot(x = "Value", y = "Feature", data = feature_imp.sort_values(by = "Value", ascending = False))
plt.tight_layout()
plt.show()